In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
path_naive = Path("../input/birdclef-2021-naive-npy")
path_naive.exists()

True

In [3]:
df_shortaudio_train = pd.read_csv(path_naive/"shortaudio_train.csv")
df_shortaudio_val = pd.read_csv(path_naive/"shortaudio_val.csv")

df_soundscape_train = pd.read_csv(path_naive/"soundscape_train.csv")
df_soundscape_val = pd.read_csv(path_naive/"soundscape_val.csv")
df_soundscape_test = pd.read_csv(path_naive/"soundscape_test.csv")

In [4]:
df_soundscape_val.head()

,row_id,site,audio_id,seconds,birds,n_birds,year,month,day,longitude,latitude,month_x,month_y,day_coarse_x,day_coarse_y,longitude_x,longitude_y,latitude_normalized
0,54955_SSW_395,SSW,54955,395,grycat,1,2017,6,17,-76.45,42.47,-1.000000e+00,1.224647e-16,-0.954139,-0.299363,0.234294,-0.972166,0.471889
1,14473_SSW_430,SSW,14473,430,nocall,0,2017,7,1,-76.45,42.47,-8.660254e-01,-5.000000e-01,0.979530,0.201299,0.234294,-0.972166,0.471889
2,26746_COR_330,COR,26746,330,nocall,0,2019,10,4,-84.51,10.12,5.000000e-01,-8.660254e-01,0.688967,0.724793,0.095672,-0.995413,0.112444
3,18003_COR_455,COR,18003,455,rucwar,1,2019,9,4,-84.51,10.12,-1.836970e-16,-1.000000e+00,0.688967,0.724793,0.095672,-0.995413,0.112444
4,14473_SSW_365,SSW,14473,365,nocall,0,2017,7,1,-76.45,42.47,-8.660254e-01,-5.000000e-01,0.979530,0.201299,0.234294,-0.972166,0.471889


In [5]:
df_shortaudio_train.head()

,primary_label,latitude,longitude,date,filename,year,month,day,npy_filename
0,cubthr,32.5839,-109.9696,2020-08-03,XC617273.ogg,2020,8,3,XC617273_100.npy
1,chbchi,35.2530,-120.8760,2020-05-21,XC561445.ogg,2020,5,21,XC561445_50.npy
2,mouela1,13.7346,-89.2796,2016-05-27,XC320698.ogg,2016,5,27,XC320698_10.npy
3,cacwre,32.7380,-112.2297,2019-09-14,XC497309.ogg,2019,9,14,XC497309_10.npy
4,burwar1,0.0287,-78.8628,2009-06-02,XC35146.ogg,2009,6,2,XC35146_80.npy


I have forgotten to add `month_x`, etc. to `shortaudio_{train,val,test}.csv`. Let's make that up.

This is not necessarily a bad thing -- By forgetting this, our `.csv` files are more ligth-weighted.

In [6]:
def cyclicize_number(number, max_, min_):
    """
    args
        number, int
            \in {min_, min_ + 1, ..., max_}
            e.g. hour => min_ = 0, max_ = 24
                 longitude => min_ = -180, max_ = 180
        max_, int
        min_, int
    return
        (x, y), tuple of float
    """
    period = max_ - min_
    theta = 2 * np.pi * (number / period)
    #theta = 2 * np.pi * ((number - min_) / period)
    x = np.cos(theta)
    y = np.sin(theta)
    return x, y

# N.B. Using the next function to deal with df_train_soundscape is
#      not efficient, since there are only 4 distinct longitudes.
def cyclicize_series(series, max_, min_):
    return list(map(lambda number: cyclicize_number(number, max_, min_), series))

In [7]:
df_shortaudio_train[["month_x", "month_y"]] = cyclicize_series(df_shortaudio_train["month"], 12, 0)
df_shortaudio_train[["day_coarse_x", "day_coarse_y"]] = cyclicize_series(df_shortaudio_train["day"], 31, 0)
df_shortaudio_train[["longitude_x", "longitude_y"]] = cyclicize_series(df_shortaudio_train["longitude"], 180, -180)
df_shortaudio_train["latitude_normalized"] = df_shortaudio_train["latitude"] / 90
df_shortaudio_train.head()

,primary_label,latitude,longitude,date,filename,year,month,day,npy_filename,month_x,month_y,day_coarse_x,day_coarse_y,longitude_x,longitude_y,latitude_normalized
0,cubthr,32.5839,-109.9696,2020-08-03,XC617273.ogg,2020,8,3,XC617273_100.npy,-5.000000e-01,-8.660254e-01,0.820763,0.571268,-0.341522,-0.939874,0.362043
1,chbchi,35.2530,-120.8760,2020-05-21,XC561445.ogg,2020,5,21,XC561445_50.npy,-8.660254e-01,5.000000e-01,-0.440394,-0.897805,-0.513182,-0.858280,0.391700
2,mouela1,13.7346,-89.2796,2016-05-27,XC320698.ogg,2016,5,27,XC320698_10.npy,-8.660254e-01,5.000000e-01,0.688967,-0.724793,0.012573,-0.999921,0.152607
3,cacwre,32.7380,-112.2297,2019-09-14,XC497309.ogg,2019,9,14,XC497309_10.npy,-1.836970e-16,-1.000000e+00,-0.954139,0.299363,-0.378321,-0.925675,0.363756
4,burwar1,0.0287,-78.8628,2009-06-02,XC35146.ogg,2009,6,2,XC35146_80.npy,-1.000000e+00,1.224647e-16,0.918958,0.394356,0.193159,-0.981167,0.000319


In [8]:
df_shortaudio_val[["month_x", "month_y"]] = cyclicize_series(df_shortaudio_val["month"], 12, 0)
df_shortaudio_val[["day_coarse_x", "day_coarse_y"]] = cyclicize_series(df_shortaudio_val["day"], 31, 0)
df_shortaudio_val[["longitude_x", "longitude_y"]] = cyclicize_series(df_shortaudio_val["longitude"], 180, -180)
df_shortaudio_val["latitude_normalized"] = df_shortaudio_val["latitude"] / 90

- Produce common columns for the two diff types of dataframes
- Build `df_train`, `df_val`, `df_test`
- Build `XX_train`, `XX_val`, `XX_test`

In [9]:
L_feature_columns = [
    "month_x",
    "month_y",
    "day_coarse_x",
    "day_coarse_y",
    "longitude_x",
    "longitude_y",
    "latitude_normalized",
]

L_common_columns = L_feature_columns + [
    "npy_filename",
    "primary_label",
]

In [10]:
df_soundscape_train["npy_filename"] = df_soundscape_train["row_id"] + ".npy"
df_soundscape_val["npy_filename"] = df_soundscape_val["row_id"] + ".npy"
df_soundscape_test["npy_filename"] = df_soundscape_test["row_id"] + ".npy"

In [11]:
df_soundscape_train.rename(
    {"birds": "primary_label"},
    axis="columns",
    inplace=True,
)
df_soundscape_val.rename(
    {"birds": "primary_label"},
    axis="columns",
    inplace=True,
)
df_soundscape_test.rename(
    {"birds": "primary_label"},
    axis="columns",
    inplace=True,
)
"primary_label" in df_soundscape_train.columns

True

In [12]:
df_train = pd.concat([
    df_shortaudio_train[L_common_columns],
    df_soundscape_train[L_common_columns],
])
df_train.shape, df_shortaudio_train.shape, df_soundscape_train.shape, len(L_common_columns)

((475238, 9), (473638, 16), (1600, 19), 9)

In [13]:
df_train.head()

,month_x,month_y,day_coarse_x,day_coarse_y,longitude_x,longitude_y,latitude_normalized,npy_filename,primary_label
0,-5.000000e-01,-8.660254e-01,0.820763,0.571268,-0.341522,-0.939874,0.362043,XC617273_100.npy,cubthr
1,-8.660254e-01,5.000000e-01,-0.440394,-0.897805,-0.513182,-0.858280,0.391700,XC561445_50.npy,chbchi
2,-8.660254e-01,5.000000e-01,0.688967,-0.724793,0.012573,-0.999921,0.152607,XC320698_10.npy,mouela1
3,-1.836970e-16,-1.000000e+00,-0.954139,0.299363,-0.378321,-0.925675,0.363756,XC497309_10.npy,cacwre
4,-1.000000e+00,1.224647e-16,0.918958,0.394356,0.193159,-0.981167,0.000319,XC35146_80.npy,burwar1


In [14]:
df_val = pd.concat([
    df_shortaudio_val[L_common_columns],
    df_soundscape_val[L_common_columns],
])
df_val.shape, df_shortaudio_val.shape, df_soundscape_val.shape, len(L_common_columns)

((203389, 9), (202989, 16), (400, 19), 9)

In [15]:
df_test = df_soundscape_test[L_common_columns]
df_test.shape

(400, 9)

Maybe we need to shuffle before assigning `df_train[L_feature_columns].value` to `XX_train`.

In [16]:
df_train = df_train.sample(frac=1)
df_val = df_val.sample(frac=1)

In [17]:
XX_train = df_train[L_feature_columns].values
XX_val = df_val[L_feature_columns].values

In [19]:
df_train

,month_x,month_y,day_coarse_x,day_coarse_y,longitude_x,longitude_y,latitude_normalized,npy_filename,primary_label
198208,6.123234e-17,1.000000e+00,-0.612106,-0.790776,0.155850,-0.987781,0.310710,XC173622_120.npy,grcfly
379454,6.123234e-17,1.000000e+00,0.151428,-0.988468,-0.543240,-0.839577,0.546796,XC156532_10.npy,bkcchi
374539,-1.000000e+00,1.224647e-16,-0.758758,0.651372,-0.451530,-0.892256,0.362226,XC571094_45.npy,buggna
304318,8.660254e-01,-5.000000e-01,-0.874347,-0.485302,0.705012,-0.709196,-0.260770,XC216466_50.npy,rcatan1
62508,8.660254e-01,5.000000e-01,-0.250653,0.968077,0.273889,-0.961761,0.125497,XC525031_45.npy,thbeup1
...,...,...,...,...,...,...,...,...,...
469592,1.000000e+00,-2.449294e-16,0.918958,-0.394356,0.994479,-0.104940,0.415203,XC297698_495.npy,categr
124945,-1.836970e-16,-1.000000e+00,-0.954139,0.299363,0.775642,-0.631173,-0.181818,XC427819_100.npy,grekis
184796,-1.000000e+00,1.224647e-16,0.151428,-0.988468,0.347016,-0.937859,0.534910,XC326711_80.npy,leafly
120517,-1.000000e+00,1.224647e-16,-0.994869,-0.101168,-0.328113,-0.944638,0.354511,XC123114_55.npy,bncfly


## Dataset Generator
Why switch to using generator? How large is our data this time? Could you make an estimate?



In [18]:
import tensorflow as tf
import tensorflow.keras as keras

In [24]:
for i in range(5):
    npy_filename_i = df_train["npy_filename"].iloc[i]
    print(npy_filename_i)

XC173622_120.npy
XC156532_10.npy
XC571094_45.npy
XC216466_50.npy
XC525031_45.npy


In [27]:
df_train["npy_filename"].head()

198208    XC173622_120.npy
379454     XC156532_10.npy
374539     XC571094_45.npy
304318     XC216466_50.npy
62508      XC525031_45.npy
Name: npy_filename, dtype: object

In [29]:
XX_train.shape, df_train.shape

((475238, 7), (475238, 9))

In [30]:
XX_train[100]

array([-1.00000000e+00,  1.22464680e-16,  3.47305253e-01, -9.37752132e-01,
       -2.73658744e-01, -9.61826851e-01,  4.53244444e-01])

In [31]:
XX_train[100].shape

(7,)

In [40]:
PATH_DATASET = Path("../input/birdclef-2021")

L_birds = [path.name for path
           in (PATH_DATASET / "train_short_audio").iterdir()]
L_birds = sorted(L_birds)
D_label_index = {label: i for i, label in enumerate(L_birds)}
D_index_label = {v: k for k, v in D_label_index.items()}

def label(series):
    #I = np.eye(len(D_label_index))
    y = np.zeros((len(series), len(D_label_index)), dtype=np.float32)
    for i, string in enumerate(series.values):
    #for i, string in enumerate(series):
        if string == "nocall":
            continue
        else:
            L_indices = [D_label_index[label] for label in string.split(" ")]
            #row_i = np.sum(I[L_indices], axis=0)
            #y[i] = row_i
            y[i, L_indices] = 1
    return y

y_train = label(df_train["primary_label"])
np.unique(np.sum(y_train, axis=-1))

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [42]:
y_train.shape

(475238, 397)

In [41]:
y_val = label(df_val["primary_label"])
#y_test = label(df_test["primary_label"])

In [43]:
def trainset_generator():
    for i in range(df_train.shape[0]):
        npy_filename_i = df_train["npy_filename"].iloc[i]
        mels_i = np.load(path_naive / f"train_npy/{npy_filename_i}").astype(np.float32, copy=False)
        image_i /= 255.0
        image_i = np.repeat(mels_i[..., np.newaxis], 3, axis=-1)  # shape: (h, w, 3)
        features_i = XX_train[i]                                  # shape: (7,)
        y_i = y_train[i]                                          # shape: (475238, 397)
        yield (image_i, features_i), y_i

def valset_generator():
    for i in range(df_val.shape[0]):
        npy_filename_i = df_val["npy_filename"].iloc[i]
        mels_i = np.load(path_naive / f"val_npy/{npy_filename_i}").astype(np.float32, copy=False)
        image_i /= 255.0
        image_i = np.repeat(mels_i[..., np.newaxis], 3, axis=-1)  # shape: (h, w, 3)
        features_i = XX_val[i]                                    # shape: (7,)
        y_i = y_val[i]
        yield (image_i, features_i), y_i